# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [8]:
# load data from database
engine = create_engine('sqlite:///DisasterResponseData.db')
df = pd.read_sql_table('Message', engine)
X = df['message']
Y = df.drop(columns=['id', 'message', 'original', 'genre'])

In [9]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [10]:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [11]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer
import re

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/edinei_santos/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/edinei_santos/nltk_data...
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/edinei_santos/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [12]:
def tokenize(text):
    """
    Normalize, tokenize, remove stop words, and lemmatize text string.
    
    Args:
        text (str): Input text string.

    Returns:
        list: List of clean tokens.
    """
    # Case normalization and punctuation removal
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())

    # Tokenization
    tokens = word_tokenize(text)

    # Stop words removal
    tokens = [t for t in tokens if t not in stopwords.words("english")]

    # Stemming and Lemmatization
    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()
    clean_tokens = [
        lemmatizer.lemmatize(stemmer.stem(token)).strip() for token in tokens
    ]

    return clean_tokens

In [13]:
# Example usage
example_texts = [
    "Some 5 or 6 mt of sorghum seed were taken when more than a dozen armed men - apparently rogue elements of the Rahanwein Resistance Army (RRA) which controls Bay and Bakol regions and parts of Middle Shabelle - held up the truck.",
    "I held my son and tried to climb up to the roof to wait for the rescue squad.",
    "(b) UNHCR expects to resume the facilitated voluntary repatriation exercise of Sierra Leonean refugees from Liberia back to Sierra Leone next week.",
    '"The pattern is always the same: steal, loot, and then set fire to everything," says the interlocutor to Misna adding that "an armed uprising is, at present, the only way for people to regain their dignity trampled by a government that is illiberal and violent."'
]

for text in example_texts:
    print(tokenize(text))

['5', '6', 'mt', 'sorghum', 'seed', 'taken', 'dozen', 'arm', 'men', 'appar', 'rogu', 'element', 'rahanwein', 'resist', 'armi', 'rra', 'control', 'bay', 'bakol', 'region', 'part', 'middl', 'shabel', 'held', 'truck']
['held', 'son', 'tri', 'climb', 'roof', 'wait', 'rescu', 'squad']
['b', 'unhcr', 'expect', 'resum', 'facilit', 'voluntari', 'repatri', 'exercis', 'sierra', 'leonean', 'refuge', 'liberia', 'back', 'sierra', 'leon', 'next', 'week']
['pattern', 'alway', 'steal', 'loot', 'set', 'fire', 'everyth', 'say', 'interlocutor', 'misna', 'ad', 'arm', 'upris', 'present', 'way', 'peopl', 'regain', 'digniti', 'trampl', 'govern', 'illiber', 'violent']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier

# Define the machine learning pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=tokenize)),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

# Display the pipeline steps
print(pipeline)


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function tokenize at 0x7fde19afeee0>)),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [15]:
# Split data into training and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [16]:
# Train the pipeline
pipeline.fit(X_train, Y_train)

/home/edinei_santos/miniconda3/envs/disaster_response_pipeline/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<function tokenize at 0x7fde19afeee0>)),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [17]:
from sklearn.metrics import classification_report

# Predict on test data
Y_pred = pipeline.predict(X_test)

# Iterate through the columns and print the classification report for each
for i, col in enumerate(Y_test.columns):
    print(f"Category: {col}")
    print(classification_report(Y_test[col], Y_pred[:, i]))
    print("\n")

Category: related
              precision    recall  f1-score   support

           0       0.71      0.43      0.53      1266
           1       0.83      0.94      0.88      3938
           2       0.46      0.33      0.38        40

    accuracy                           0.81      5244
   macro avg       0.67      0.57      0.60      5244
weighted avg       0.80      0.81      0.80      5244



Category: request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      4349
           1       0.84      0.48      0.61       895

    accuracy                           0.90      5244
   macro avg       0.87      0.73      0.78      5244
weighted avg       0.89      0.90      0.88      5244



Category: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00        26

    accuracy                           1.00      5244
   macro avg      

/home/edinei_santos/miniconda3/envs/disaster_response_pipeline/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/edinei_santos/miniconda3/envs/disaster_response_pipeline/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/edinei_santos/miniconda3/envs/disaster_response_pipeline/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` pa

### 6. Improve your model
Use grid search to find better parameters. 

In [27]:
from sklearn.model_selection import GridSearchCV

# Define parameters for Grid Search
parameters = {
    'tfidf__max_df': [0.75, 1.0],
    'clf__estimator__min_samples_split': [2, 4]
}

# Create Grid Search object
cv = GridSearchCV(pipeline, param_grid=parameters, cv=3, verbose=3, n_jobs=1)

In [28]:
# Train the pipeline with Grid Search
cv.fit(X_train, Y_train)

# Display a message indicating training is complete
print("Grid Search training complete.")

Fitting 3 folds for each of 4 candidates, totalling 12 fits


/home/edinei_santos/miniconda3/envs/disaster_response_pipeline/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 1/3] END clf__estimator__min_samples_split=2, tfidf__max_df=0.75;, score=0.260 total time= 3.8min


/home/edinei_santos/miniconda3/envs/disaster_response_pipeline/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 2/3] END clf__estimator__min_samples_split=2, tfidf__max_df=0.75;, score=0.258 total time= 3.9min


...

[CV 2/3] END clf__estimator__min_samples_split=4, tfidf__max_df=1.0;, score=0.252 total time= 3.1min


/home/edinei_santos/miniconda3/envs/disaster_response_pipeline/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 3/3] END clf__estimator__min_samples_split=4, tfidf__max_df=1.0;, score=0.266 total time= 3.1min


/home/edinei_santos/miniconda3/envs/disaster_response_pipeline/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Grid Search training complete.


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [29]:
from sklearn.metrics import accuracy_score

# Predict on test data
Y_pred = cv.predict(X_test)

# Initialize lists to store the results
accuracy_list = []
precision_list = []
recall_list = []

# Iterate through the columns and print the classification report for each
for i, col in enumerate(Y_test.columns):
    print(f"Category: {col}")
    report = classification_report(Y_test[col], Y_pred[:, i], output_dict=True)
    print(classification_report(Y_test[col], Y_pred[:, i]))
    print("\n")

    # Append the results to the lists
    accuracy_list.append(accuracy_score(Y_test[col], Y_pred[:, i]))
    precision_list.append(report['weighted avg']['precision'])
    recall_list.append(report['weighted avg']['recall'])

Category: related
              precision    recall  f1-score   support

           0       0.72      0.43      0.54      1266
           1       0.83      0.94      0.89      3938
           2       0.43      0.38      0.40        40

    accuracy                           0.82      5244
   macro avg       0.66      0.58      0.61      5244
weighted avg       0.80      0.82      0.80      5244



Category: request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      4349
           1       0.84      0.49      0.62       895

    accuracy                           0.90      5244
   macro avg       0.87      0.74      0.78      5244
weighted avg       0.89      0.90      0.89      5244



Category: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00        26

    accuracy                           1.00      5244
   macro avg      

/home/edinei_santos/miniconda3/envs/disaster_response_pipeline/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/edinei_santos/miniconda3/envs/disaster_response_pipeline/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/edinei_santos/miniconda3/envs/disaster_response_pipeline/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` pa

              precision    recall  f1-score   support

           0       0.87      0.99      0.93      4549
           1       0.45      0.03      0.05       695

    accuracy                           0.87      5244
   macro avg       0.66      0.51      0.49      5244
weighted avg       0.82      0.87      0.81      5244



Category: infrastructure_related
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      4916
           1       0.00      0.00      0.00       328

    accuracy                           0.94      5244
   macro avg       0.47      0.50      0.48      5244
weighted avg       0.88      0.94      0.91      5244



Category: transport
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      5004
           1       0.65      0.07      0.13       240

    accuracy                           0.96      5244
   macro avg       0.81      0.53      0.55      5244
weighted avg     

/home/edinei_santos/miniconda3/envs/disaster_response_pipeline/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/edinei_santos/miniconda3/envs/disaster_response_pipeline/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/edinei_santos/miniconda3/envs/disaster_response_pipeline/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` pa

In [30]:
# Calculate the average accuracy, precision, and recall
average_accuracy = np.mean(accuracy_list)
average_precision = np.mean(precision_list)
average_recall = np.mean(recall_list)

print(f"Average Accuracy: {average_accuracy:.4f}")
print(f"Average Precision: {average_precision:.4f}")
print(f"Average Recall: {average_recall:.4f}")

Average Accuracy: 0.9488
Average Precision: 0.9383
Average Recall: 0.9488


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [43]:
# POS Tagging
from sklearn.base import BaseEstimator, TransformerMixin
import nltk
from nltk import pos_tag
from sklearn.preprocessing import OneHotEncoder

nltk.download('averaged_perceptron_tagger')

class POSTagEncoder(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.encoder = OneHotEncoder(handle_unknown='ignore')
    
    def fit(self, x, y=None):
        pos_tags = self._extract_pos_tags(x)
        self.encoder.fit(pos_tags)
        return self
    
    def transform(self, x):
        pos_tags = self._extract_pos_tags(x)
        return self.encoder.transform(pos_tags)
    
    def _extract_pos_tags(self, x):
        pos_tags = [pos_tag(word_tokenize(text)) for text in x]
        pos_tags = [[' '.join([tag for _, tag in tags])] for tags in pos_tags]
        return pos_tags


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/edinei_santos/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [44]:
# Named Entity Recognition
from nltk.chunk import ne_chunk
from nltk.tokenize import word_tokenize


nltk.download('maxent_ne_chunker')
nltk.download('words')

class NEREncoder(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.encoder = OneHotEncoder(handle_unknown='ignore')
    
    def fit(self, x, y=None):
        named_entities = self._extract_named_entities(x)
        self.encoder.fit(named_entities)
        return self
    
    def transform(self, x):
        named_entities = self._extract_named_entities(x)
        return self.encoder.transform(named_entities)
    
    def _extract_named_entities(self, x):
        named_entities = [ne_chunk(pos_tag(word_tokenize(text))) for text in x]
        named_entities = [[' '.join([chunk.label() if hasattr(chunk, 'label') else '' for chunk in entity])] for entity in named_entities]
        return named_entities


[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/edinei_santos/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /home/edinei_santos/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [45]:
from sklearn.pipeline import FeatureUnion
from xgboost import XGBClassifier

# New pipeline with additional features and using XGBoost
pipeline = Pipeline([
    ('features', FeatureUnion([
        ('tfidf', TfidfVectorizer(tokenizer=tokenize)),
        ('pos_tags', POSTagEncoder()),
        ('named_entities', NEREncoder())
    ])),
    ('clf', MultiOutputClassifier(XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')))
])

In [46]:
# Define parameters for Grid Search
parameters = {
    'features__tfidf__max_df': [0.75, 1.0],
    'features__tfidf__ngram_range': [(1, 1)],
    'clf__estimator__n_estimators': [50],
    'clf__estimator__learning_rate': [0.1]
}

In [47]:
# Create Grid Search object with n_jobs=1 to disable parallel processing
cv = GridSearchCV(pipeline, param_grid=parameters, cv=3, verbose=3)

# Train the pipeline with Grid Search
cv.fit(X_train, Y_train)

# Display a message indicating training is complete
print("Grid Search training complete.")

Fitting 3 folds for each of 2 candidates, totalling 6 fits


/home/edinei_santos/miniconda3/envs/disaster_response_pipeline/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 1/3] END clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=50, features__tfidf__max_df=0.75, features__tfidf__ngram_range=(1, 1);, score=0.248 total time= 3.7min


/home/edinei_santos/miniconda3/envs/disaster_response_pipeline/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 2/3] END clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=50, features__tfidf__max_df=0.75, features__tfidf__ngram_range=(1, 1);, score=0.245 total time= 3.9min


...

[CV 2/3] END clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=50, features__tfidf__max_df=1.0, features__tfidf__ngram_range=(1, 1);, score=0.245 total time= 3.6min


/home/edinei_santos/miniconda3/envs/disaster_response_pipeline/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV 3/3] END clf__estimator__learning_rate=0.1, clf__estimator__n_estimators=50, features__tfidf__max_df=1.0, features__tfidf__ngram_range=(1, 1);, score=0.256 total time= 3.5min


/home/edinei_santos/miniconda3/envs/disaster_response_pipeline/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Grid Search training complete.


In [48]:
# Predict on test data
Y_pred = cv.predict(X_test)

# Initialize lists to store the results
accuracy_list = []
precision_list = []
recall_list = []

# Iterate through the columns and print the classification report for each
for i, col in enumerate(Y_test.columns):
    print(f"Category: {col}")
    report = classification_report(Y_test[col], Y_pred[:, i], output_dict=True)
    print(classification_report(Y_test[col], Y_pred[:, i]))
    print("\n")

    # Append the results to the lists
    accuracy_list.append(accuracy_score(Y_test[col], Y_pred[:, i]))
    precision_list.append(report['weighted avg']['precision'])
    recall_list.append(report['weighted avg']['recall'])

Category: related
              precision    recall  f1-score   support

           0       0.79      0.14      0.24      1266
           1       0.78      0.99      0.87      3938
           2       0.50      0.05      0.09        40

    accuracy                           0.78      5244
   macro avg       0.69      0.39      0.40      5244
weighted avg       0.78      0.78      0.71      5244



Category: request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      4349
           1       0.82      0.48      0.61       895

    accuracy                           0.89      5244
   macro avg       0.86      0.73      0.77      5244
weighted avg       0.89      0.89      0.88      5244



Category: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00        26

    accuracy                           1.00      5244
   macro avg      

/home/edinei_santos/miniconda3/envs/disaster_response_pipeline/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/edinei_santos/miniconda3/envs/disaster_response_pipeline/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/edinei_santos/miniconda3/envs/disaster_response_pipeline/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` pa

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      5184
           1       0.91      0.17      0.28        60

    accuracy                           0.99      5244
   macro avg       0.95      0.58      0.64      5244
weighted avg       0.99      0.99      0.99      5244



Category: refugees
              precision    recall  f1-score   support

           0       0.98      0.99      0.98      5073
           1       0.61      0.25      0.36       171

    accuracy                           0.97      5244
   macro avg       0.79      0.62      0.67      5244
weighted avg       0.96      0.97      0.96      5244



Category: death
              precision    recall  f1-score   support

           0       0.98      0.99      0.99      5007
           1       0.80      0.51      0.63       237

    accuracy                           0.97      5244
   macro avg       0.89      0.75      0.81      5244
weighted avg       0.97      0.97  

/home/edinei_santos/miniconda3/envs/disaster_response_pipeline/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/edinei_santos/miniconda3/envs/disaster_response_pipeline/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/edinei_santos/miniconda3/envs/disaster_response_pipeline/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` pa

In [49]:
# Calculate the average accuracy, precision, and recall
average_accuracy = np.mean(accuracy_list)
average_precision = np.mean(precision_list)
average_recall = np.mean(recall_list)

print(f"Average Accuracy: {average_accuracy:.4f}")
print(f"Average Precision: {average_precision:.4f}")
print(f"Average Recall: {average_recall:.4f}")

Average Accuracy: 0.9494
Average Precision: 0.9426
Average Recall: 0.9494


### 9. Export your model as a pickle file

In [50]:
import pickle

# Save the trained model to a pickle file
model_filepath = 'disaster_response_model.pkl'
with open(model_filepath, 'wb') as file:
    pickle.dump(cv, file)

print(f"Model saved to {model_filepath}")


Model saved to disaster_response_model.pkl


### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.